In [ ]:
import torch
import os
import albumentations as A

from torch.utils.data import DataLoader
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from dataset import PlatesDataset, collate_fn
from fasterrcnn_trainer import ModelTrainer

In [ ]:
import wandb

run = wandb.init(project="car-plates", entity="ai-talent-itmo", reinit=True)

In [ ]:
## Опеределяем конфигурацию обучения

Config = {
    # detection train dataset params
    'root_path': '../vkcv2022-contest-02-carplates/data/',
    'json_path': '../vkcv2022-contest-02-carplates/data/train.json',
    'val_size': 0.2,
    
    # detection training params
    'model_name': 'faster_rcnn_v5',
    'device': 'cuda:2',
    'num_epochs': 20,
    'lr': 3e-4,
    'batch_size': 16,
    'num_workers': 8
}


In [ ]:
## Добавляем аугментацию для расширения датасета

train_transform = A.Compose([A.HorizontalFlip(p=0.25),
                            A.Rotate(limit=(-20, 20), p=0.25),
                            A.ColorJitter(brightness=0.3, contrast=0.2, saturation=0.2, hue=0.05, p=0.25)],
                            bbox_params={
                                        'format': 'pascal_voc',
                                        'label_fields': ['labels']
                                        })

trainset = PlatesDataset(root_path=Config['root_path'], json_path=Config['json_path'],
                         sample_type='train', val_size=Config['val_size'], transform=train_transform)

valset = PlatesDataset(root_path=Config['root_path'], json_path=Config['json_path'],
                       sample_type='val', val_size=Config['val_size'])

print('Train size:', len(trainset))
print('Val size:', len(valset))

In [ ]:
## Запускаем обучение модели

faster_rcnn_model = fasterrcnn_resnet50_fpn_v2(pretrained=True)
in_features = faster_rcnn_model.roi_heads.box_predictor.cls_score.in_features
faster_rcnn_model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)

    
for param in faster_rcnn_model.backbone.body.parameters():
    param.requires_grad = False
    
for param in faster_rcnn_model.backbone.body.layer4.parameters():
    param.requires_grad = True
    
for name, param in faster_rcnn_model.named_parameters():
    print(name, param.requires_grad)
    

faster_rcnn_model.to(Config['device'])
optimizer = torch.optim.AdamW(faster_rcnn_model.parameters(), lr=Config['lr'])


train_loader = DataLoader(trainset, batch_size=Config['batch_size'], shuffle=True, 
                          collate_fn=collate_fn, num_workers=Config['num_workers'])
val_loader = DataLoader(valset, batch_size=Config['batch_size'], shuffle=False, 
                        collate_fn=collate_fn, num_workers=Config['num_workers'])


scheduller = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer, max_lr=Config['lr'], 
                                                 steps_per_epoch=len(train_loader), 
                                                epochs=Config['num_epochs'], pct_start=0.05, anneal_strategy='cos')

wandb.config = {
    "learning_rate": Config['lr'],
    "epochs": Config['num_epochs'],
    "batch_size": Config['batch_size'],
    "model_name": Config['model_name'],
    "scheduler": "OneCycleLR",
    "optimizer": "AdamW"
}

In [ ]:
model_trainer = ModelTrainer(model=faster_rcnn_model, model_name=Config['model_name'], 
                             train_loader=train_loader, val_loader=val_loader,
                            optimizer=optimizer, scheduller=scheduller, device=Config['device'], run=run,
                            val_classes=[1])

model_trainer.train_net(num_epochs=Config['num_epochs'])

Обучили модель деткции Faster_rcnn на 20 эпох:

Лучшая метрика MAP50_95 = 77.9%     

Логи обучения: 'Logs/faster_rcnn_v5.txt'